## Regression with Quantum Kernel and Support Vector Machine


This notebook provides a comprehensive guide on defining quantum kernels using qiskit-machine-learning and applying them to regression tasks.

We utilize the well-known Diabetes dataset, which serves as a common benchmark in machine learning due to its straightforward yet informative structure. This dataset is an effective foundation for demonstrating the capabilities of Support Vector Machines (SVM) and comparing them with Quantum Kernel Support Vector Machines (QSVM) in the context of regression.


The notebook is structured into two sections:
- Section 1: Regression with Classical SVM
- Section 2: Regression with Quantum SVM



In [1]:
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from qiskit.circuit.library import ZZFeatureMap, TwoLocal, RealAmplitudes

from qiskit_machine_learning.algorithms import QSVR
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit.primitives import StatevectorSampler

from sklearn.svm import SVR
from sklearn.metrics import root_mean_squared_error, mean_absolute_percentage_error


### Quantum Kernels

Quantum kernel machine learning primarly involves utilizing quantum feature maps to apply the kernel trick. This process involves mapping a classical feature vector $\vec{x}$ to a Hilbert space using a quantum feature map $\phi(\vec{x})$. This is represented as:

$K_{ij} = \left| \langle \phi(\vec{x}_i)| \phi(\vec{x}_j) \rangle \right|^{2}$

Here 
* $K_{ij}$ represents the kernel matrix
* $\vec{x}_i$ and $\vec{x}_j$ are $n$ dimensional inputs vectors.
* $\phi(\vec{x})$ denotes the quantum feature map.
* $\left| \langle a|b \rangle \right|^{2}$ is the overlap between two quantum states $a$ and $b$.

Quantum kernels can seamlessly integrate with common classical kernel learning algorithms, such as Support Vector Machines (SVMs), as demonstrated in the examples below.

In [2]:
#Import data
data = load_diabetes()

df_features = pd.DataFrame(data.data, columns=data.feature_names)
df_target = pd.DataFrame(data.target)
df_features.head(5)

data.data = data.data[:, 0:4] # We will only use 4 features


We normalize the data

In [3]:
# Scale the data
train_X, test_X, train_y, test_y = train_test_split(
    data.data, data.target, train_size=0.8, shuffle=True)

scaler_features = MinMaxScaler()
X_train_scaled = scaler_features.fit_transform(train_X)
X_test_scaled = scaler_features.transform(test_X)

num_features = train_X.shape[1] #num features

# Section 1: Regression with Classical SVM


In [4]:
#Train and test SVM
svr = SVR()
_ = svr.fit(X_train_scaled, train_y) 

train_pred = svr.predict(X_train_scaled)
test_pred = svr.predict(X_test_scaled)


metrics_svm = {'rmse_train': root_mean_squared_error(train_pred, train_y),
                    'rmse_test': root_mean_squared_error(test_pred, test_y),
                    'mape_train': mean_absolute_percentage_error(train_pred, train_y),
                    'mape_test': mean_absolute_percentage_error(test_pred, test_y),
                    }
print('Results Classical SVM ')
print(f'RMSE train: { metrics_svm['rmse_train']:.4f}')
print(f'RMSE test: {  metrics_svm['rmse_test']:.4f}')
print(f'MAPE train: { metrics_svm['mape_train']:.4f}')
print(f'MAPE test: {  metrics_svm['mape_test']:.4f}')


Results Classical SVM 
RMSE train: 71.9026
RMSE test: 75.6352
MAPE train: 0.4259
MAPE test: 0.4728


# Section 2: Regression with Quantum SVM

In the next step, we will create a quantum kernel. To accomplish this, we first need to map the features using `feature_map`, specifically the ZZFeatureMap. Following this, we will utilize ComputeUncompute to calculate the `fidelity`, which will be used in our FidelityQuantumKernel—the quantum kernel we will employ.

In [5]:
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)
sampler = StatevectorSampler()
fidelity = ComputeUncompute(sampler=sampler)
fidelity_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)


Finally, we fit the QSVR to the data and evaluate the regression models employed.

In [6]:
# Create a QSVR instance
qsvr = QSVR(quantum_kernel=fidelity_kernel)
qsvr.fit(X_train_scaled, train_y)

QSVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
     max_iter=-1,
     quantum_kernel=<qiskit_machine_learning.kernels.fidelity_quantum_kernel.FidelityQuantumKernel object at 0x7f20384d5310>,
     shrinking=True, tol=0.001, verbose=False)

In [7]:
train_pred = qsvr.predict(X_train_scaled)
test_pred = qsvr.predict(X_test_scaled)


metrics_estimator = {'rmse_train': root_mean_squared_error(train_pred, train_y),
                    'rmse_test': root_mean_squared_error(test_pred, test_y),
                    'mape_train': mean_absolute_percentage_error(train_pred, train_y),
                    'mape_test': mean_absolute_percentage_error(test_pred, test_y),
                    }
print('Results Quantum SVM:  ')
print(f'RMSE train: { metrics_estimator['rmse_train']:.4f}')
print(f'RMSE test: { metrics_estimator['rmse_test']:.4f}')
print(f'MAPE train: { metrics_estimator['mape_train']:.4f}')
print(f'MAPE test: { metrics_estimator['mape_test']:.4f}')



Results Quantum SVM:  
RMSE train: 76.3589
RMSE test: 79.5922
MAPE train: 0.4601
MAPE test: 0.5031


## Bonus

Additionally, we can use the quantum kernel directly with the SVR defined in scikit-learn, eliminating the need to utilize the QSVR from qiskit-machine-learning. To achieve this, we simply need to provide the quantum kernel as a callable to the SVR. Let's take a look at how this is done:

In [8]:
#Define again the feature map and quantum-kernel
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)
sampler = StatevectorSampler()
fidelity = ComputeUncompute(sampler=sampler)
fidelity_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)

#This is the new important step, where the quantum kernel is directly pass to SVR
qkernel_svc = SVR(kernel=fidelity_kernel.evaluate)

qkernel_svc.fit(X_train_scaled, train_y) 

train_pred = qkernel_svc.predict(X_train_scaled)
test_pred = qkernel_svc.predict(X_test_scaled)


metrics_svm = {'rmse_train': root_mean_squared_error(train_pred, train_y),
                    'rmse_test': root_mean_squared_error(test_pred, test_y),
                    'mape_train': mean_absolute_percentage_error(train_pred, train_y),
                    'mape_test': mean_absolute_percentage_error(test_pred, test_y),
                    }
print('Results Quantum SVM:  ')
print(f'RMSE train: { metrics_svm['rmse_train']:.4f}')
print(f'RMSE test: {  metrics_svm['rmse_test']:.4f}')
print(f'MAPE train: { metrics_svm['mape_train']:.4f}')
print(f'MAPE test: {  metrics_svm['mape_test']:.4f}')


Results Quantum SVM:  
RMSE train: 76.4027
RMSE test: 79.6536
MAPE train: 0.4608
MAPE test: 0.5038
